## Storage

In [ ]:
# Count the occurrences of each name and filter where counts are greater than 1
repeated_names_a = debar['Name'].value_counts()
repeated_names_a = repeated_names_a[repeated_names_a > 1].index.tolist()

# Print the list of repeated names
print(repeated_names_a)


In [ ]:
# Group by 'Name' and count the number of entries for each group
repeated_names_b = debar.groupby('Name').size()

# Filter groups where the count is greater than 1
repeated_names_b = repeated_names_b[repeated_names_b > 1].index.tolist()

# Print the list of repeated names
print(repeated_names_b)


In [ ]:
# New column for repeated employer names
debar['is_repeated'] = debar['Name'].duplicated(keep=False)

# Print repeat rows 
print(debar[debar['is_repeated']])

# Subset to repeat rows 
mult_debar = debar[debar['is_repeated']]

# Head and shape of the new df
mult_debar

In [ ]:

# Adding a helper column 'violation_number' to enumerate violations for each employer
mult_debar['violation_number'] = mult_debar.groupby(['Name', 'City, State']).cumcount() + 1

# Pivot the DataFrame to wide format
mult_debar_wide = mult_debar.pivot_table(
    index=['Name', 'City, State'],
    columns='violation_number',
    values='Start date',
    aggfunc='first'
)

# Rename columns for clarity
mult_debar_wide.columns = [f'start_date_viol{num}' for num in mult_debar_wide.columns]

# Reset index to make 'Name' and 'City, State' regular columns
mult_debar_wide.reset_index(inplace=True)

mult_debar_wide

# # Display the head of the new DataFrame and its shape
# print(mult_debar_wide.head())
# print(mult_debar_wide.shape)


In [ ]:

# Create a unique identifier based on 'Name', 'City, State', and 'Start date'
mult_debar['unique_id'] = mult_debar['Name'] + mult_debar['City, State'] + mult_debar['Start date'].astype(str)

# Flag duplicates based on the unique identifier
mult_debar['is_duplicate'] = mult_debar.duplicated('unique_id', keep=False)

# Separate the data into duplicates and unique violation cases
duplicates = mult_debar[mult_debar['is_duplicate']]
unique_violations = mult_debar[~mult_debar['is_duplicate']]


duplicates
unique_violations

# # Print results
# print("Duplicates:")
# print(duplicates.head())
# print("Unique Violations:")
# print(unique_violations.head())

# # Print the shape of the resulting DataFrames
# print("Shape of Duplicates DataFrame:", duplicates.shape)
# print("Shape of Unique Violations DataFrame:", unique_violations.shape)
